# **Financial Report Analysis**

**Objective**

Develop an AI-powered Financial Image Scans
 ● Scanthedocument and then identify,capture the relevant images you want to use for the prompts.

 ● Extracts and highlights critical information from financial document image, such as key financial metrics, trends, and risks.

 ● Summarize complex financial content into concise, easy-to-understand insights tailored to the user’s requirements (e.g., investors, analysts, or auditors).

 ● Enables users to focus on actionable insights, reducing time spent on document review and improving decision-making efficiency.

 ● Is able to analyze the graphs and charts present inside the documents and give outputs
 based on it.

In [ ]:
!pip install openai --quiet

In [ ]:
from PIL import Image

In [ ]:
from getpass import getpass
OpenAI_key=getpass('Enter your OpenAI key')

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OpenAI_key

In [ ]:
from openai import OpenAI
import os
MODEL="gpt-4o"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

To provide context-aware, actionable insights specific to the user’s goals, making
 them less effective for quick and informed decision-making

In [ ]:
from IPython.display import Image, display, Audio, Markdown
import base64

IMAGE_PATH1 = "F:\\Pinnacle AI Plus\\Prompt Engineering Essentials\\Assignment\\Consolidated_statement.png"
IMAGE_PATH2 = "F:\\Pinnacle AI Plus\\Prompt Engineering Essentials\\Assignment\\Figures_at_Glance.png"
Image_list=[IMAGE_PATH1,IMAGE_PATH2]
for image in Image_list:
    display(Image(image))
# Preview image for context
#display(Image(IMAGE_PATH1))
#display(Image(IMAGE_PATH2))

In [ ]:
import base64
from PIL import Image
import os
import io
from functools import wraps

def image_extract_decorator(image_paths, extraction_prompt):
    """Decorator that takes image paths and an extraction prompt, encodes images to base64, and passes them to the function."""
    def decorator(func):
        @wraps(func)
        def wrapper(user_prompt, client, model, *args, **kwargs):
            encoded_images = []
            for image_path in image_paths:
                try:
                    if not os.path.exists(image_path):
                        print(f"File not found: {image_path}")
                        continue
                    with Image.open(image_path) as img:
                        buffer = io.BytesIO()
                        img.save(buffer, format="PNG")
                        buffer.seek(0)
                        encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
                        encoded_images.append({"path": image_path, "base64": encoded_image})
                        #print(f"Encoded: {image_path}")
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
            
            if not encoded_images:
                input_dir = os.path.dirname(image_paths[0]) if image_paths else ""
                if input_dir and os.path.exists(input_dir):
                    print("Directory contents:", os.listdir(input_dir))
                else:
                    print(f"Input directory does not exist: {input_dir}")
                return {"error": "No valid images found"}
            
            return func(user_prompt, client, model, encoded_images=encoded_images, extraction_prompt=extraction_prompt, *args, **kwargs)
        return wrapper
    return decorator

In [ ]:
@image_extract_decorator(
    image_paths=[
        r"F:\Pinnacle AI Plus\Prompt Engineering Essentials\Assignment\Consolidated_statement.png",
        r"F:\Pinnacle AI Plus\Prompt Engineering Essentials\Assignment\Figures_at_Glance.png"
    ],
    extraction_prompt="Extract key financial metrics (revenue, net income, EPS) and summarize any charts or tables from the images."
)
def extract_financial_data(user_prompt, client, model, encoded_images=None, extraction_prompt=None):
    """Function to process a prompt and base64-encoded images using an AI model API."""
    try:
        # Combine user prompt with extraction prompt
        combined_prompt = f"{user_prompt}\nExtraction Instructions: {extraction_prompt}"

        messages=[ 
{"role": "system", "content": "You are an advanced AI-powered Financial Document Analysis System. Your expertise lies in interpreting complex financial data from scanned images and providing actionable insights for various financial stakeholders."},
{"role": "user", "content": [
                {"type": "text", "text": combined_prompt}
            ] + [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img['base64']}"}}
                for img in encoded_images
            ]},
{"role": "user", "content": [
    {"type": "text", "text": """Your primary function is to enhance {Image_data} and decision-making efficiency by automating critical information extraction, analysis, and summarization from image-based financial documents. Your analysis should be grounded in the provided document content and tailored to specific user requirements.
• Scan the image content to identify and capture all relevant data (including graphs, charts, and tables) that contain financial data or insights.
• Extract and highlight critical information such as key financial metrics (e.g., revenue, net income, earnings per share (EPS)), identified trends (e.g., growth patterns, market shifts, performance over time, compared to previous periods), and potential risks (e.g., debt levels, liquidity issues, volatility indicators).
• Analyze all embedded graphs and charts to derive quantitative and qualitative outputs. Translate visual data points into descriptive insights and numerical values where appropriate.
• Summarize the complex financial content into concise, easy-to-understand insights. This summary must be tailored to the specific user role (e.g., 'investor', 'analyst', or 'auditor') which will be indicated.
    ◦ Specifically, for the summary, you must:
        ▪ Highlight key performance metrics such as revenue, net income, earnings per share (EPS), and growth trends compared to previous periods.
        ▪ Summarize key financial statements like the balance sheet, income statement, and cash flow statement to reflect the company's assets, liabilities, equity, revenue, expenses, and cash flows.
        ▪ Provide insights into operational performance, market conditions, and management's commentary on future outlooks, risks, and strategic priorities.
        ▪ Include financial ratios (such as profitability, liquidity, and leverage ratios) and any notable events (such as acquisitions or major partnerships).
        ▪ Discuss any risks, challenges, and external factors that might impact the business, along with the company’s approach to managing those risks.
        ▪ If applicable, include shareholder-related information such as dividends or stock performance.
• Ensure the output focuses exclusively on actionable insights, enabling users to quickly grasp key takeaways.

[Output Format]:
<Key Financial Metrics>
	Here include 5(at most) key financial metrics in bullet points
<Income and Expenses>
	Here include 5(at most) income and expense events in bullet points
<Balance Sheet>
	Here include 5(atmost) balance sheet highlights in bullet points
<Credit Quality>
	Here include 5(at most) credit quality highlights in bullet points
<Strategic and Operational Updates>
	Here include 5(at most) strategic and operational updates in bullet points
<Market Conditions and Outlook>
	Here include 5(at most) Market Conditions and Outlook highlighst in bullet points
<Summary>
	Here summarize the decision with 5 points and also state what would be the best decision to take, either good to invest in or not a good-time to invest in.(Choose only from the given two options)
"""}
    ]}
    
    ]
        
        # Make API call
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.0,
        )
        
        # Extract response content
        result = {
            "prompt": combined_prompt,
            "image_count": len(encoded_images),
            "images_processed": [{"path": img["path"], "base64_length": len(img["base64"])} for img in encoded_images],
            "api_response": response.choices[0].message.content if response.choices else "No response from API"
        }
        #print(f"API call successful for {len(encoded_images)} images")
        return result
    
    except Exception as e:
        print(f"API call failed: {e}")
        return {"error": str(e), "prompt": combined_prompt, "image_count": len(encoded_images)}

In [ ]:
if __name__ == "__main__":
    
    user_prompt = "Analyze the financial documents and provide insights for investors."
    result = extract_financial_data(user_prompt, client, MODEL)

In [ ]:
display(Markdown(result['api_response']))